# Imports

In [1]:
import classification as mt

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # adapt plots for retina displays

/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


# Load model, dataset and brain area masks

In [2]:
!ls

APTOS2019-TAME	   docker_image     params.json  requirements.txt  utils
classification.py  misc		    __pycache__  self_supervised   wandb
defaults	   models_and_misc  README.md	 train_tame.ipynb


In [3]:
net, train_loader, num_classes = mt.get_jupyter_items()

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
=> no checkpoint found at '/home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino'


In [4]:
# Important: Set model to eval before using any interpretation methods
net.eval()

Classifier(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (norm): Layer

# Build TAME model (from scratch)

In [5]:
from tame.utilities.pl_module import TAMELIT

version = "TAME"
model = net.cpu()
layers = [
    "backbone.blocks.9",
    "backbone.blocks.10",
    "backbone.blocks.11",
]
epochs = 8

pl_model = TAMELIT(
    model_name="vit_b_16",
    model=model,
    layers=layers,
    attention_version=version,
    train_method="new",
    lr=0.001,
    epochs=epochs,
    num_classes=num_classes,
    eval_length="long",
    LeRF=True
)

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp018xo_9b
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp018xo_9b/_remote_module_non_scriptable.py


In [6]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="APTOS2019-TAME", name="random-new")
trainer = pl.Trainer(
    enable_checkpointing=True,
    max_epochs=epochs,
    logger=wandb_logger,
    log_every_n_steps=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
import torch

torch.set_float32_matmul_precision("medium")

## Train

In [8]:
print(next(iter(train_loader))[0][0].shape)

torch.Size([3, 224, 224])


In [9]:
trainer.fit(pl_model, train_loader)

/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name           | Type               | Params
------------------------------------------------------
0 | generic        | Generic            | 22.1 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
451 K     Trainable params
21.7 M    Non-trainable params
22.1 M    Total params
88.477    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.


## Evaluate with AD and IC metrics

In [10]:
_, test_loader, _ = mt.get_jupyter_items(train=False)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
=> no checkpoint found at '/home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino'


In [11]:
# trainer.logger = CSVLogger("logs", name="3D-TAME")
trainer.test(pl_model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

[[ 1.75793374e+00  2.34075442e-01 -8.83394718e-01  2.16356084e-01
  -1.37033546e+00]
 [ 5.27328849e-01  2.22958699e-01  3.94428782e-02  1.65705547e-01
  -2.10500196e-01]
 [ 1.59318745e+00  8.21854025e-02  6.27030283e-02 -4.35738862e-01
  -3.58068794e-02]
 [ 1.25030708e+00  2.96656072e-01 -3.69876117e-01 -2.83080161e-01
  -1.12882781e+00]
 [ 9.58813429e-01  3.29177499e-01  8.62265378e-02 -2.08141506e-02
  -3.79507005e-01]
 [ 1.43073583e+00 -2.38260105e-01  3.12410623e-01 -1.50349632e-01
  -3.90650094e-01]
 [ 2.41130769e-01  4.48796451e-01 -7.13155270e-01  8.63197148e-02
  -8.49144876e-01]
 [ 1.20727766e+00  7.27692842e-01 -1.20844710e+00  4.86570835e-01
  -2.19230279e-01]
 [ 5.87782502e-01 -3.20067704e-01 -5.48376083e-01 -2.24930421e-01
  -1.00863802e+00]
 [ 1.28859663e+00  6.58828467e-02 -3.19137901e-01  1.07471883e-01
  -1.25532305e+00]
 [ 2.02432680e+00  9.18436348e-01 -6.21809602e-01  2.28746638e-01
  -6.15949810e-01]
 [ 7.84182608e-01  1.25382125e+00 -6.22891307e-01 -2.12558255e-01

[{}]

In [12]:
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/Accuracy,█▃▁▁▂▂▃▂▁▃▁▁▂▂▁▃▂▂▁▃▁▅▂▁▅▂▁▂▃▁▂▂▁▅▁▁▁▁▃▁
training/CE,█▁▂▂▂▂▁▂▁▁▂▂▂▁▂▃▁▂▂▁▂▂▁▁▃▂▂▁▂▂▂▂▂▁▂▂▂▂▁▂
training/Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/Mean,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/Var,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,7
trainer/global_step,319
training/Accuracy,0.0
training/CE,2.72994


## Train deit-small TAME on ImageNet

In [1]:
%cd ..

/home/ntrougkas/Documents/3D-TAME


In [2]:
import os
from pathlib import Path

import timm
from dotenv import load_dotenv

from tame.utilities.pl_module import TAMELIT, LightnightDataset

load_dotenv("/home/ntrougkas/Documents/3D-TAME/.env")

epochs = 8
old_model = TAMELIT(
    model=timm.create_model("deit_small_patch16_224", pretrained=True),
    model_name="vit_b_16",
    layers=["blocks.9", "blocks.10", "blocks.11"],
    attention_version="TAME",
    normalized_data=True,
    lr=0.001,
    epochs=epochs,
    eval_length="short",
)

imagenet_data = LightnightDataset(
    dataset_path=Path(os.getenv("DATA", "./")),
    datalist_path=Path(os.getenv("LIST", "./")),
    model="vit_b_16",
    batch_size=32,
)

INFO:timm.models.helpers:Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth)
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpkn_v02dm
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpkn_v02dm/_remote_module_non_scriptable.py


In [3]:
import torch

torch.set_float32_matmul_precision("medium")

In [4]:
from lightning.pytorch.loggers import WandbLogger
import lightning.pytorch as pl

wandb_logger = WandbLogger(project="T-TAME-small")
trainer = pl.Trainer(enable_checkpointing=True, max_epochs=epochs, logger=wandb_logger)
trainer.fit(old_model, imagenet_data)
trainer.test(old_model, imagenet_data)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name           | Type               | Params
------------------------------------------------------
0 | generic        | Generic            | 23.6 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
1.6 M     Trainable params
22.1 M    Non-trainable params
23.6 M    Total params
94.598    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          AD 100%          │    12.129423141479492     │
│          AD 15%           │     77.4417495727539      │
│          AD 50%           │     32.22406768798828     │
│          IC 100%          │    18.700000762939453     │
│          IC 15%           │     4.700000286102295     │
│          IC 50%           │           22.0            │
└───────────────────────────┴───────────────────────────┘

[{'AD 100%': 12.129423141479492,
  'IC 100%': 18.700000762939453,
  'AD 50%': 32.22406768798828,
  'IC 50%': 22.0,
  'AD 15%': 77.4417495727539,
  'IC 15%': 4.700000286102295}]

In [5]:
import wandb

wandb.alert(
    title="Training finished",
    text="Training on deit-small finished, can finish medical transformers task",
)
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


AD 100%,▁
AD 15%,▁
AD 50%,▁
IC 100%,▁
IC 15%,▁
IC 50%,▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training/Accuracy,▄▂▁▂▅▅▄▂▇▅▆▃▂▆▃▃▂█▅▅▇▆▅▅▄▆▅▂▅▅▅▃▂▅▅▅▅▇▅▅
training/CE,▆▇█▆▄▃▅▅▃▅▂▄▇▃▇▄▅▁▃▅▄▃▂▃▄▂▃▇▂▃▃▄▆▂▄▃▅▃▄▃
training/Loss,█▇▇▅▄▅▄▅▃▄▂▄▇▃▆▃▄▁▃▄▄▃▂▃▃▂▃▆▂▂▃▃▅▂▃▂▃▂▃▂


Training complete, now we can finetune the trained TAME model on APTOS

# Finetune TAME on APTOS

In [13]:
!ls

APTOS2019-TAME	   docker_image     params.json  requirements.txt  utils
classification.py  misc		    __pycache__  self_supervised   wandb
defaults	   models_and_misc  README.md	 train_tame.ipynb


In [17]:
%cd medical_transformers

[Errno 2] No such file or directory: 'medical_transformers'
/home/ntrougkas/Documents/3D-TAME/medical_transformers


In [5]:
import classification as mt

net, train_loader, num_classes = mt.get_jupyter_items()
net.eval()
from tame.utilities.pl_module import TAMELIT

version = "TAME"
model = net.cpu()
layers = [
    "backbone.blocks.9.norm1",
    "backbone.blocks.10.norm1",
    "backbone.blocks.11.norm1",
]
epochs = 8

pl_model = TAMELIT(
    model_name="vit_b_16",
    model=model,
    layers=layers,
    attention_version=version,
    train_method="new",
    lr=0.001,
    epochs=epochs,
    num_classes=num_classes,
    eval_length="long",
    LeRF=True
)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
=> no checkpoint found at '/home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino'


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpbvnsgu7e
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpbvnsgu7e/_remote_module_non_scriptable.py


Use new fusion module

In [6]:
import timm

pretrained_model = TAMELIT.load_from_checkpoint(
    "/home/ntrougkas/Documents/3D-TAME/T-TAME-small/1k8nfdz4/checkpoints/epoch=1-step=80074.ckpt",
    model=timm.create_model("deit_small_patch16_224", pretrained=True),
)
pretrained_model.generic.attn_mech[1].fuser = pl_model.generic.attn_mech[1].fuser
pl_model.generic.attn_mech[1] = pretrained_model.generic.attn_mech[1]

INFO:timm.models.helpers:Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth)


Train

In [7]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
import torch

torch.set_float32_matmul_precision("medium")
wandb_logger = WandbLogger(project="APTOS2019-TAME", name="pretrained")
trainer = pl.Trainer(
    enable_checkpointing=True,
    max_epochs=epochs,
    logger=wandb_logger,
    log_every_n_steps=1,
)
trainer.fit(pl_model, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name           | Type               | Params
------------------------------------------------------
0 | generic        | Generic            | 22.1 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
451 K     Trainable params
21.7 M    Non-trainable params
22.1 M    Total params
88.477    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.


Test

In [8]:
_, test_loader, _ = mt.get_jupyter_items(train=False)
trainer.test(pl_model, test_loader)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
=> no checkpoint found at '/home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino'


/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

[[-7.73047268e-01  1.40248621e+00  1.50417164e-02 -8.70132267e-01
   6.69835269e-01]
 [-1.24547124e+00  7.37201750e-01 -2.49262750e-01 -1.22420669e+00
   6.95458472e-01]
 [-1.30725026e+00  7.82420039e-01  2.37378255e-02 -8.50661397e-01
   1.11446095e+00]
 [-2.44237110e-01  5.14640749e-01 -2.34528959e-01 -3.20525914e-01
   6.44534409e-01]
 [-1.01196241e+00  1.18972099e+00 -2.74237275e-01 -7.60018170e-01
   1.06485379e+00]
 [-1.45021462e+00  1.19202264e-01 -3.83206666e-01 -6.02957070e-01
   9.66707051e-01]
 [-1.07928574e+00  3.32882017e-01 -2.99953401e-01 -2.22310320e-01
   9.97650683e-01]
 [-8.50571811e-01  1.35888898e+00  3.56467441e-02 -5.96588612e-01
   3.52394015e-01]
 [-1.55541313e+00  6.68659210e-01 -2.72296593e-02 -6.41906261e-01
   4.12098944e-01]
 [-1.26848733e+00  1.25614607e+00 -1.67396247e-01 -6.71370149e-01
   1.31549263e+00]
 [-7.18298376e-01  4.94090647e-01 -4.07367870e-02 -7.92512357e-01
   6.41974211e-01]
 [-1.25006902e+00  1.29013932e+00  2.29787305e-02 -6.38633609e-01

[{}]

In [9]:
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/Accuracy,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/CE,▁▆██████████████████████████████████████
training/Loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/Mean,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training/Var,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,7
trainer/global_step,319
training/Accuracy,0.0
training/CE,2.86149


## Do the same with new fuser module, retaining the older one and adapting it to new output classes

In [1]:
import classification as mt

net, train_loader, num_classes = mt.get_jupyter_items()
net.eval()
from tame.utilities.pl_module import TAMELIT

version = "TAME"
model = net.cpu()
layers = [
    "backbone.blocks.9",
    "backbone.blocks.10",
    "backbone.blocks.11",
]
epochs = 8

pl_model = TAMELIT(
    model_name="vit_b_16",
    model=model,
    layers=layers,
    attention_version=version,
    train_method="new",
    lr=0.0001,
    epochs=epochs,
    num_classes=num_classes,
    eval_length="long",
)

/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp7rkablpl
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp7rkablpl/_remote_module_non_scriptable.py


Proxy(getattr_5) Proxy(getattr_6)
Proxy(getattr_5) Proxy(getattr_6)
Proxy(getattr_5) Proxy(getattr_6)
Proxy(getattr_5) Proxy(getattr_6)


Adapt existing fusion module

In [2]:
import timm
from torch import nn

pretrained_model = TAMELIT.load_from_checkpoint(
    "/home/ntrougkas/Documents/3D-TAME/T-TAME-small/vkep09vs/checkpoints/epoch=7-step=320296.ckpt",
    model=timm.create_model("deit_small_patch16_224", pretrained=True),
)
pretrained_model.generic.attn_mech[1].fuser = nn.Sequential(
    pretrained_model.generic.attn_mech[1].fuser,
    nn.Conv2d(
        in_channels=1000,
        out_channels=num_classes,
        kernel_size=1,
        padding=0,
        bias=True,
    ),
)
pl_model.generic.attn_mech[1] = pretrained_model.generic.attn_mech[1]

INFO:timm.models.helpers:Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth)


Train

In [3]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import LearningRateMonitor

import torch

torch.set_float32_matmul_precision("medium")
wandb_logger = WandbLogger(project="APTOS2019-TAME", name="pretrained")
trainer = pl.Trainer(
    enable_checkpointing=True,
    max_epochs=epochs,
    logger=wandb_logger,
    log_every_n_steps=1,
    callbacks=[
        LearningRateMonitor(
            logging_interval="step", log_momentum=True, log_weight_decay=True
        )
    ],
)
trainer.fit(pl_model, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name           | Type               | Params
------------------------------------------------------
0 | generic        | Generic            | 23.3 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
1.6 M     Trainable params
21.7 M    Non-trainable params
23.3 M    Total params
93.086    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.


In [6]:
a = torch.randn(4, 4)
a

tensor([[-0.9214,  0.5485,  0.5358,  0.5041],
        [-0.0627, -1.0408,  0.4921,  0.2686],
        [ 0.6596, -0.0087, -0.2173, -0.6798],
        [-0.4622, -1.5323, -0.9715, -0.5757]])

In [8]:
torch.argmax(a, dim=1)

tensor([1, 2, 0, 0])

Test

In [4]:
_, test_loader, _ = mt.get_jupyter_items(train=False)
trainer.test(pl_model, test_loader)

Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]

In [5]:
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LR/pg1,▁▂▂▃▃▄▅▆▇▇███████▇▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
LR/pg1-momentum,██▇▇▆▅▄▃▂▂▁▁▁▁▁▁▁▂▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇▇█████
LR/pg1-weight_decay,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
LR/pg2,▁▂▂▃▃▄▅▆▇▇███████▇▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
LR/pg2-momentum,██▇▇▆▅▄▃▂▂▁▁▁▁▁▁▁▂▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇▇█████
LR/pg2-weight_decay,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training/Accuracy,▆█▄▄▅▅▆▃▄▅▅▃▃▆▄▄▃▃▄▁▆▁▃▅▅▅▄▆▅▅▂▃▆▅▃▅▅▄▃▇
training/CE,▁▆▇▇▆▇▆▇▇▆▆▇▆▅▇▆▇▇▅█▅▇▇▆▆▆▆▆▆▆█▇▆▇▇▆▅▆▇▅
training/Loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


# Calculate AD and IC for competing methods

## First apply on gradcam

In [1]:
import classification as mt

net, test_loader, _ = mt.get_jupyter_items(train=False)
net.eval()
from tame.utilities.comp_module import CompareModel
import torch
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

model = net

pl_model = CompareModel(
    mdl_name="vit",
    name="gradcam",
    raw_model=model,
    target_layers=[model.backbone.blocks[11]],
    eval_length="long",
)


wandb_logger = WandbLogger(project="APTOS2019-TAME", name="gradcam")
torch.set_float32_matmul_precision("medium")
trainer = pl.Trainer(logger=wandb_logger)

/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


Initialising Dataloaders . . .
The default metric has been set to : cohen_kappa
Initialising Model . . .
Initialising Optimization methods . . 
LARS OPTIMIZER:  INACTIVE 
Loading model from /home/ntrougkas/Documents/3D-TAME/medical_transformers/models_and_misc/CVAMD2021/checkpoints/APTOS2019_trans-pre_dino


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpuu12s0r9
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpuu12s0r9/_remote_module_non_scriptable.py
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [2]:
trainer.test(pl_model, test_loader)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ntrougkas/Documents/3D-TAME/.venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

torch.Size([64, 197, 384]) torch.Size([1, 197, 384])
1
torch.Size([64, 197, 384]) torch.Size([1, 197, 384])


OutOfMemoryError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 0; 7.79 GiB total capacity; 6.94 GiB already allocated; 35.75 MiB free; 7.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
wandb_logger.finalize("success")
wandb_logger.experiment.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


## Do the same for the rest of the methods

In [12]:
from tame.utilities.comp_module import CompareModel
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

methods = [
    "scorecam",
    "gradcam++",
    "ablationcam",
]

net, test_loader, _ = mt.get_jupyter_items(train=False)
net.eval()
model = net

for method_name in methods:
    pl.utilities.memory.garbage_collection_cuda()
    torch.cuda.empty_cache()
    pl_model = CompareModel(
        mdl_name="vit",
        name="gradcam",
        raw_model=model,
        target_layers=[model.backbone.blocks[11]],
        eval_length="long",
    )
    wandb_logger = WandbLogger(project="APTOS2019-TAME", name=method_name)
    torch.set_float32_matmul_precision("medium")
    trainer = pl.Trainer(logger=wandb_logger)
    trainer.test(pl_model, test_loader)
    wandb_logger.finalize("success")
    wandb_logger.experiment.finish()

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmphild9ozc
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmphild9ozc/_remote_module_non_scriptable.py
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marios1861. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

1
2


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [ ]:
# Set up a binary mask for each brain area based on the AAL atlas.
# This is required for brain area occlusion and to determine the relevance per area.
# The AAL atlas can for example be retrieved from here: https://github.com/neurolabusc/MRIcron/blob/master/templates/aal.nii.gz
brain_map = utils.load_nifti("data/aal.nii.gz")
brain_areas = np.unique(brain_map)[1:]  # omit background

area_masks = []
for area in tqdm_notebook(brain_areas):
    area_mask = np.zeros_like(brain_map)
    area_mask[brain_map == area] = 1
    area_mask = utils.resize_image(
        area_mask, test_dataset.image_shape(), interpolation=0
    )
    area_masks.append(area_mask)

area_names = [
    "Precentral_L",
    "Precentral_R",
    "Frontal_Sup_L",
    "Frontal_Sup_R",
    "Frontal_Sup_Orb_L",
    "Frontal_Sup_Orb_R",
    "Frontal_Mid_L",
    "Frontal_Mid_R",
    "Frontal_Mid_Orb_L",
    "Frontal_Mid_Orb_R",
    "Frontal_Inf_Oper_L",
    "Frontal_Inf_Oper_R",
    "Frontal_Inf_Tri_L",
    "Frontal_Inf_Tri_R",
    "Frontal_Inf_Orb_L",
    "Frontal_Inf_Orb_R",
    "Rolandic_Oper_L",
    "Rolandic_Oper_R",
    "Supp_Motor_Area_L",
    "Supp_Motor_Area_R",
    "Olfactory_L",
    "Olfactory_R",
    "Frontal_Sup_Medial_L",
    "Frontal_Sup_Medial_R",
    "Frontal_Med_Orb_L",
    "Frontal_Med_Orb_R",
    "Rectus_L",
    "Rectus_R",
    "Insula_L",
    "Insula_R",
    "Cingulum_Ant_L",
    "Cingulum_Ant_R",
    "Cingulum_Mid_L",
    "Cingulum_Mid_R",
    "Cingulum_Post_L",
    "Cingulum_Post_R",
    "Hippocampus_L",
    "Hippocampus_R",
    "ParaHippocampal_L",
    "ParaHippocampal_R",
    "Amygdala_L",
    "Amygdala_R",
    "Calcarine_L",
    "Calcarine_R",
    "Cuneus_L",
    "Cuneus_R",
    "Lingual_L",
    "Lingual_R",
    "Occipital_Sup_L",
    "Occipital_Sup_R",
    "Occipital_Mid_L",
    "Occipital_Mid_R",
    "Occipital_Inf_L",
    "Occipital_Inf_R",
    "Fusiform_L",
    "Fusiform_R",
    "Postcentral_L",
    "Postcentral_R",
    "Parietal_Sup_L",
    "Parietal_Sup_R",
    "Parietal_Inf_L",
    "Parietal_Inf_R",
    "SupraMarginal_L",
    "SupraMarginal_R",
    "Angular_L",
    "Angular_R",
    "Precuneus_L",
    "Precuneus_R",
    "Paracentral_Lobule_L",
    "Paracentral_Lobule_R",
    "Caudate_L",
    "Caudate_R",
    "Putamen_L",
    "Putamen_R",
    "Pallidum_L",
    "Pallidum_R",
    "Thalamus_L",
    "Thalamus_R",
    "Heschl_L",
    "Heschl_R",
    "Temporal_Sup_L",
    "Temporal_Sup_R",
    "Temporal_Pole_Sup_L",
    "Temporal_Pole_Sup_R",
    "Temporal_Mid_L",
    "Temporal_Mid_R",
    "Temporal_Pole_Mid_L",
    "Temporal_Pole_Mid_R",
    "Temporal_Inf_L",
    "Temporal_Inf_R",
    "Cerebelum_Crus1_L",
    "Cerebelum_Crus1_R",
    "Cerebelum_Crus2_L",
    "Cerebelum_Crus2_R",
    "Cerebelum_3_L",
    "Cerebelum_3_R",
    "Cerebelum_4_5_L",
    "Cerebelum_4_5_R",
    "Cerebelum_6_L",
    "Cerebelum_6_R",
    "Cerebelum_7b_L",
    "Cerebelum_7b_R",
    "Cerebelum_8_L",
    "Cerebelum_8_R",
    "Cerebelum_9_L",
    "Cerebelum_9_R",
    "Cerebelum_10_L",
    "Cerebelum_10_R",
    "Vermis_1_2",
    "Vermis_3",
    "Vermis_4_5",
    "Vermis_6",
    "Vermis_7",
    "Vermis_8",
    "Vermis_9",
    "Vermis_10",
]

# Merge left and right areas.
merged_area_names = [name[:-2] for name in area_names[:108:2]] + area_names[108:]

In [ ]:
def get_relevance_per_area(relevance_map, normalize=True):
    relevances = np.zeros(len(area_masks))
    for i, area_mask in enumerate(area_masks):
        relevances[i] = np.sum(relevance_map * area_mask)
    if normalize:
        relevances /= relevances.sum()  # make all areas sum to 1

    # Merge left and right areas.
    merged_relevances = np.concatenate(
        [relevances[:108].reshape(-1, 2).sum(1), relevances[108:]]
    )

    return sorted(
        zip(merged_area_names, merged_relevances), key=lambda x: x[1], reverse=True
    )

# Relevance heatmaps for single images

## Raw image

In [ ]:
which = 1

image_tensor = test_dataset[which][0]
raw_image = test_dataset.get_raw_image(which)

utils.plot_slices(raw_image, num_slices=7)

## Sensitivity Analysis

In [ ]:
relevance_map_backprop = interpretation.sensitivity_analysis(
    net, image_tensor, cuda=True, verbose=True
)

In [ ]:
utils.plot_slices(
    raw_image,
    overlay=relevance_map_backprop[0],
    overlay_vmax=np.percentile(relevance_map_backprop, 99.9),
    overlay_cmap=utils.alpha_to_red_cmap,
)

In [ ]:
get_relevance_per_area(relevance_map_backprop[0])[:10]

## Guided Backpropagation

In [ ]:
relevance_map_guided = interpretation.guided_backprop(
    net, image_tensor, cuda=True, verbose=True
)

In [ ]:
utils.plot_slices(
    raw_image,
    overlay=relevance_map_guided[0],
    overlay_vmax=np.percentile(relevance_map_guided, 99.9),
)

In [ ]:
get_relevance_per_area(relevance_map_guided[0])[:10]

## Occlusion

In [ ]:
relevance_map_occlusion = interpretation.occlusion(
    net, image_tensor, size=40, stride=25, cuda=True, resize=True, verbose=True
)

In [ ]:
utils.plot_slices(
    raw_image, overlay=relevance_map_occlusion, overlay_cmap=utils.alpha_to_red_cmap
)

In [ ]:
utils.plot_slices(
    raw_image, overlay=relevance_map_occlusion, overlay_cmap=utils.alpha_to_red_cmap
)

In [ ]:
get_relevance_per_area(relevance_map_occlusion)[:10]

In [ ]:
# Plot occlusion patch on image.
occluded_image = raw_image[:, 114, :].copy()
size = 40
occluded_image[50 : 50 + size, 70 : 70 + size] = 0
plt.imshow(occluded_image, cmap="gray")
plt.axis("off")

## Area Occlusion

In [ ]:
relevance_map_area_occlusion = interpretation.area_occlusion(
    net, image_tensor, area_masks, cuda=True, verbose=True
)

In [ ]:
utils.plot_slices(
    raw_image,
    overlay=relevance_map_area_occlusion,
    overlay_cmap=utils.alpha_to_red_cmap,
)  # , overlay_vmin=0, overlay_vmax=1)

In [ ]:
get_relevance_per_area(relevance_map_area_occlusion)[:10]

## Grad-CAM
Note: Grad-CAM doesn't work on this data. It's only included here for completeness, but I couldn't figure out if there's a bug in the implementation or if the method is not applicable to this kind of data and/or model.

In [ ]:
# TODO: Something is apparently wrong with gradcam, try to fix it. Could also simply be because feature maps of last layer are just 4x5x4, so very small.

In [ ]:
relevance_map_grad_cam = interpretation.grad_cam(
    net, image_tensor, cuda=True, resize=True, interpolation=0, verbose=True
)

In [ ]:
utils.plot_slices(
    raw_image, overlay=relevance_map_grad_cam * test_dataset.std + test_dataset.mean
)

In [ ]:
utils.plot_slices(raw_image, overlay=relevance_map_grad_cam)

In [ ]:
utils.plot_slices(relevance_map_grad_cam)

In [ ]:
get_relevance_per_area(relevance_map_grad_cam)[:10]

# Relevance heatmaps averaged over the dataset
Plot an average relevance map of all Alzheimer and all control patients (in the test set). These are the kind of plots that are included in the paper.

In [ ]:
# If None, average over the entire dataset, otherwise pick a number of samples.
num_samples = None

# The background over which to plot the heatmaps.
bg = test_dataset.mean

## Backpropagation

In [ ]:
(
    avg_relevance_map_AD_backprop,
    avg_relevance_map_NC_backprop,
    avg_relevance_map_all_backprop,
) = interpretation.average_over_dataset(
    interpretation.sensitivity_analysis,
    net,
    test_dataset,
    num_samples=num_samples,
    seed=0,
    show_progress=True,
    cuda=True,
)

In [ ]:
utils.plot_slices(
    bg,
    overlay=avg_relevance_map_AD_backprop[0],
    overlay_vmax=np.percentile(avg_relevance_map_AD_backprop, 99.9),
)

In [ ]:
get_relevance_per_area(avg_relevance_map_AD_backprop)[:10]

In [ ]:
utils.plot_slices(
    bg,
    overlay=avg_relevance_map_NC_backprop[0],
    overlay_vmax=np.percentile(avg_relevance_map_NC_backprop, 99.9),
)

In [ ]:
get_relevance_per_area(avg_relevance_map_NC_backprop)[:10]

## Guided Backpropagation

In [ ]:
(
    avg_relevance_map_AD_guided,
    avg_relevance_map_NC_guided,
    avg_relevance_map_all_guided,
) = interpretation.average_over_dataset(
    interpretation.guided_backprop,
    net,
    test_dataset,
    num_samples=num_samples,
    seed=0,
    show_progress=True,
    cuda=True,
)

In [ ]:
utils.plot_slices(
    bg,
    overlay=avg_relevance_map_AD_guided[0],
    overlay_vmax=np.percentile(avg_relevance_map_AD_guided, 99.9),
)

In [ ]:
get_relevance_per_area(avg_relevance_map_AD_guided)[:10]

In [ ]:
utils.plot_slices(
    bg,
    overlay=avg_relevance_map_NC_guided[0],
    overlay_vmax=np.percentile(avg_relevance_map_NC_guided, 99.9),
)

In [ ]:
get_relevance_per_area(avg_relevance_map_NC_guided)[:10]

## Occlusion

In [ ]:
(
    avg_relevance_map_AD_occlusion,
    avg_relevance_map_NC_occlusion,
    avg_relevance_map_all_occlusion,
) = interpretation.average_over_dataset(
    interpretation.occlusion,
    net,
    test_dataset,
    num_samples=num_samples,
    show_progress=True,
    size=40,
    stride=40,
    cuda=True,
)

In [ ]:
utils.plot_slices(
    bg, overlay=avg_relevance_map_AD_occlusion, overlay_cmap=utils.alpha_to_red_cmap
)

In [ ]:
get_relevance_per_area(avg_relevance_map_AD_occlusion)[:10]

In [ ]:
utils.plot_slices(
    bg, overlay=avg_relevance_map_NC_occlusion, overlay_cmap=utils.alpha_to_red_cmap
)

In [ ]:
get_relevance_per_area(avg_relevance_map_NC_occlusion)[:10]

## Area Occlusion

In [ ]:
(
    avg_relevance_map_AD_area_occlusion,
    avg_relevance_map_NC_area_occlusion,
    avg_relevance_map_all_area_occlusion,
) = interpretation.average_over_dataset(
    interpretation.area_occlusion,
    net,
    test_dataset,
    num_samples=num_samples,
    seed=0,
    show_progress=True,
    cuda=True,
    area_masks=area_masks,
)

In [ ]:
utils.plot_slices(
    bg,
    overlay=avg_relevance_map_AD_area_occlusion,
    overlay_cmap=utils.alpha_to_red_cmap,
)

In [ ]:
get_relevance_per_area(avg_relevance_map_AD_area_occlusion)[:10]

In [ ]:
utils.plot_slices(bg, overlay=avg_relevance_map_NC_area_occlusion)

In [ ]:
get_relevance_per_area(avg_relevance_map_NC_area_occlusion)[:10]

## Grad-CAM

In [ ]:
(
    avg_relevance_map_AD_grad_cam,
    avg_relevance_map_NC_grad_cam,
    avg_relevance_map_all_grad_cam,
) = average_over_dataset(
    grad_cam,
    net,
    val_dataset,
    num_samples=10,
    show_progress=True,
    resize=True,
    interpolation=0,
    cuda=True,
)

In [ ]:
utils.plot_slices(mask, overlay=avg_relevance_map_AD_grad_cam)

In [ ]:
get_relevance_per_area(avg_relevance_map_AD_grad_cam)[:10]

In [ ]:
utils.plot_slices(mask, overlay=avg_relevance_map_NC_grad_cam)

In [ ]:
get_relevance_per_area(avg_relevance_map_NC_grad_cam)[:10]

## Save average heatmaps

In [ ]:
# Save heatmaps to file.
# np.savez_compressed('output/relevance_maps_final_compressed.npz', **{
#    'avg_relevance_map_AD_backprop': avg_relevance_map_AD_backprop,
#    'avg_relevance_map_AD_guided': avg_relevance_map_AD_guided,
#    'avg_relevance_map_AD_occlusion': avg_relevance_map_AD_occlusion,
#    'avg_relevance_map_AD_area_occlusion': avg_relevance_map_AD_area_occlusion,
#    'avg_relevance_map_AD_grad_cam': avg_relevance_map_AD_grad_cam,

#    'avg_relevance_map_NC_backprop': avg_relevance_map_NC_backprop,
#    'avg_relevance_map_NC_guided': avg_relevance_map_NC_guided,
#    'avg_relevance_map_NC_occlusion': avg_relevance_map_NC_occlusion,
#    'avg_relevance_map_NC_area_occlusion': avg_relevance_map_NC_area_occlusion,
#    'avg_relevance_map_NC_grad_cam': avg_relevance_map_NC_grad_cam
# })

# Figures and tables for paper

In [ ]:
# Load heatmaps from file.
loaded_heatmaps = np.load("output/relevance_maps_final_compressed.npz")
avg_relevance_map_AD_backprop = loaded_heatmaps["avg_relevance_map_AD_backprop"]
avg_relevance_map_AD_guided = loaded_heatmaps["avg_relevance_map_AD_guided"]
avg_relevance_map_AD_occlusion = loaded_heatmaps["avg_relevance_map_AD_occlusion"]
avg_relevance_map_AD_area_occlusion = loaded_heatmaps[
    "avg_relevance_map_AD_area_occlusion"
]
avg_relevance_map_AD_grad_cam = loaded_heatmaps["avg_relevance_map_AD_grad_cam"]
avg_relevance_map_NC_backprop = loaded_heatmaps["avg_relevance_map_NC_backprop"]
avg_relevance_map_NC_guided = loaded_heatmaps["avg_relevance_map_NC_guided"]
avg_relevance_map_NC_occlusion = loaded_heatmaps["avg_relevance_map_NC_occlusion"]
avg_relevance_map_NC_area_occlusion = loaded_heatmaps[
    "avg_relevance_map_NC_area_occlusion"
]
avg_relevance_map_NC_grad_cam = loaded_heatmaps["avg_relevance_map_NC_grad_cam"]

## Figure 1 (Average heatmaps)

In [ ]:
def plot_column(
    struct_arr,
    axes,
    title,
    cmap="gray",
    vmin=None,
    vmax=None,
    overlay=None,
    overlay_cmap=utils.alpha_to_red_cmap,
    overlay_vmin=None,
    overlay_vmax=None,
):
    if vmin is None:
        vmin = struct_arr.min()
    if vmax is None:
        vmax = struct_arr.max()
    if overlay_vmin is None and overlay is not None:
        overlay_vmin = overlay.min()
    if overlay_vmax is None and overlay is not None:
        overlay_vmax = overlay.max()
    print(vmin, vmax, overlay_vmin, overlay_vmax)

    offset = 80
    num_slices = len(axes)

    intervals = (np.asarray(struct_arr.shape) - 2 * offset) / (num_slices - 1)

    axis = 1
    axis_label = "y"

    # for axes_column in zip(titles):

    for i, ax in enumerate(axes):
        # print(axis_label, 'plotting slice', i_slice)
        i_slice = int(np.round(offset + i * intervals[axis]))

        plt.sca(ax)
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        # plt.axis('off')
        plt.imshow(
            sp.ndimage.rotate(np.take(struct_arr, i_slice, axis=axis), 90),
            vmin=vmin,
            vmax=vmax,
            cmap=cmap,
            interpolation=None,
        )
        plt.text(
            0.03,
            0.97,
            "{}".format(i_slice),
            color="white",
            horizontalalignment="left",
            verticalalignment="top",
            transform=ax.transAxes,
        )

        if overlay is not None:
            plt.imshow(
                sp.ndimage.rotate(np.take(overlay, i_slice, axis=axis), 90),
                cmap=overlay_cmap,
                vmin=overlay_vmin,
                vmax=overlay_vmax,
                interpolation=None,
            )

        if i == 0:
            plt.title(title + "\n")
            # plt.ylabel('Backpropagation', rotation=0, size='large')
            # ax.get_yaxis().set_label_coords(-0.5, 0.5)

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(11, 7.5))
axes = axes.T

# bg = mask
bg = test_dataset.mean
vmax = 550  # make the background brain a bit lighter

plot_column(
    bg,
    axes[0],
    "Sensitivity Analysis\n(Backpropagation)",
    overlay=avg_relevance_map_NC_backprop[0],
    overlay_vmax=np.percentile(avg_relevance_map_NC_backprop, 99.9),
    vmax=vmax,
)
plot_column(
    bg,
    axes[1],
    "Guided\nBackpropagation",
    overlay=avg_relevance_map_NC_guided[0],
    overlay_vmax=np.percentile(avg_relevance_map_NC_guided, 99.9),
    vmax=vmax,
)
plot_column(
    bg, axes[2], "Occlusion\n", overlay=avg_relevance_map_NC_occlusion, vmax=vmax
)
plot_column(
    bg,
    axes[3],
    "Brain Area\nOcclusion",
    overlay=avg_relevance_map_NC_area_occlusion,
    vmax=vmax,
)
# plot_column(bg, axes[4], 'Grad-CAM\n', overlay=avg_relevance_map_AD_grad_cam, vmax=vmax)
plt.tight_layout()

# plt.savefig('heatmaps-nc.png', dpi=300)

## Table 1 (Most relevant brain areas)

In [ ]:
most_relevant_areas_per_method = [
    get_relevance_per_area(avg_relevance_map_AD_backprop)[:4],
    get_relevance_per_area(avg_relevance_map_AD_guided)[:4],
    get_relevance_per_area(avg_relevance_map_AD_occlusion)[:4],
    get_relevance_per_area(avg_relevance_map_AD_area_occlusion)[:4]  # ,
    # get_relevance_per_area(avg_relevance_map_AD_grad_cam)[:4]
]

In [ ]:
lines = []
for i in range(len(most_relevant_areas_per_method[0])):
    line = " & ".join(
        [
            "{} ({:.1f} \\%)".format(areas[i][0].replace("_", ""), areas[i][1] * 100)
            for areas in most_relevant_areas_per_method
        ]
    )
    lines.append(line)
print(" \\\\ \n".join(lines))  # output in latex format

## Table 2 (Euclidean distances between heatmaps)

In [ ]:
relevance_maps_AD = [
    avg_relevance_map_AD_backprop,
    avg_relevance_map_AD_guided,
    avg_relevance_map_AD_occlusion,
    avg_relevance_map_AD_area_occlusion,
]
relevance_maps_NC = [
    avg_relevance_map_NC_backprop,
    avg_relevance_map_NC_guided,
    avg_relevance_map_NC_occlusion,
    avg_relevance_map_NC_area_occlusion,
]
names = ["backprop ", "guided   ", "occlusion", "area occl"]

In [ ]:
scale_factor = 1e-4

print(
    "Euclidean distance between average heatmaps for AD / NC samples in {}".format(
        scale_factor
    )
)
print()

print("\t\t" + "\t".join(names))

for a_AD, a_NC, a_name in zip(relevance_maps_AD, relevance_maps_NC, names):
    print(a_name, end="\t")
    for b_AD, b_NC, b_name in zip(relevance_maps_AD, relevance_maps_NC, names):
        print(
            "{:.2f} / {:.2f}".format(
                interpretation.heatmap_distance(a_AD, b_AD) / scale_factor,
                interpretation.heatmap_distance(a_NC, b_NC) / scale_factor,
            ),
            end="\t",
        )
    print()

In [ ]:
print(
    "Euclidean distance between AD and NC heatmaps for each method in {}".format(
        scale_factor
    )
)
print()

print("\t".join(names))
for a, b in zip(relevance_maps_AD, relevance_maps_NC):
    print(
        "{:.2f}".format(interpretation.heatmap_distance(a, b) / scale_factor),
        end="\t\t",
    )

# Export to Animation and MRIcron

In [ ]:
anim = utils.animate_slices(
    test_dataset.mean,
    overlay=avg_relevance_map_AD_guided[0],
    overlay_vmax=np.percentile(avg_relevance_map_AD_guided, 99.9),
    axis=2,
    reverse_direction=True,
    interval=70,
)
plt.close()  # suppress plot output

In [ ]:
plt.rcParams[
    "animation.ffmpeg_path"
] = "/home/johannesr/ffmpeg-3.4.1-64bit-static/ffmpeg"

In [ ]:
# Display the animation inline.
from IPython.display import HTML

HTML(anim.to_html5_video())

In [ ]:
anim.save("data/anim-guided.gif")

In [ ]:
# You can also save the relevance map in NIFTI format and load it as an overlay in MRIcron (https://www.nitrc.org/projects/mricron) later.
# utils.save_nifti('guided_backprop.nii', relevance_map_guided[0])